根据电信客户信息（telco-churn.csv），建立一个模型，预测哪些客户会流失。
该数据来自IBM，包含多个字段：费用、使用期限、流量信息以及一个表明客户是否会流失的变量（churn）等。

>- (1) 读入数据，删去不必要的变量customerID
>- (2) 将TotalCharges由字符串转换为数值。提示：
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'],errors='coerce’)
说明：若errors设为“coerce”，无效的解析将设置为NaN。
>- (3) 将所有分类变量转换为标签编码。
>- (4) 随机打乱数据。拆分出X，y。8：2 划分数据集为训练集和测试集。
>- (5）训练一个XGBoost模型，令其有1000棵树,学习率为0.01,max_depth=4，输出测试集的分类性能报告。保存模型为json文件。

In [20]:
import pandas as pd
import numpy as np

# 读取数据
data = pd.read_csv(r'D:\2024\课内\AI homework\XGBoost\telco-churn.csv')

print(data.head())
data.info()

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

>- (4) 随机打乱数据。拆分出X，y。8：2 划分数据集为训练集和测试集。
>- (5）训练一个XGBoost模型，令其有1000棵树,学习率为0.01,max_depth=4，输出测试集的分类性能报告。保存模型为json文件。

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 步骤 1: 删除 'customerID' 列
data = data.drop(columns=['customerID'])

# 步骤 2: 将 'TotalCharges' 转换为数值，强制错误为 NaN
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

# 处理转换后 'TotalCharges' 中可能的缺失值
data['TotalCharges'].fillna(0, inplace=True)

# 步骤 3: 编码分类变量
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# 步骤 4: 拆分特征 (X) 和目标 (y)
X = data.drop(columns=['Churn'])
y = data['Churn']

# 随机打乱并拆分数据 (80% 训练集, 20% 测试集)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 验证拆分结果
X_train.shape, X_test.shape, y_train.shape, y_test.shape


C:\Users\Administrator\AppData\Local\Temp\ipykernel_34088\2307985332.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['TotalCharges'].fillna(0, inplace=True)


((5634, 19), (1409, 19), (5634,), (1409,))

In [26]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import json

# 调整模型参数以加快训练速度
xgb_model = XGBClassifier(n_estimators=1000, learning_rate=0.01, max_depth=4, random_state=42, use_label_encoder=False)

# 训练模型
xgb_model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = xgb_model.predict(X_test)

# 生成分类报告
report = classification_report(y_test, y_pred, output_dict=True)

# 将模型保存为 JSON 文件
model_json = xgb_model.get_booster().save_config()
with open('D:/2024/课内/AI homework/XGBoost/xgboost_model.json', 'w') as f:
    json.dump(model_json, f)

# 打印分类报告
print(report)


d:\py\Anaconda3\envs\mathmodeling\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:15:37] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'0': {'precision': 0.8420107719928187, 'recall': 0.9054054054054054, 'f1-score': 0.8725581395348837, 'support': 1036.0}, '1': {'precision': 0.6677966101694915, 'recall': 0.5281501340482574, 'f1-score': 0.5898203592814372, 'support': 373.0}, 'accuracy': 0.8055358410220014, 'macro avg': {'precision': 0.7549036910811551, 'recall': 0.7167777697268314, 'f1-score': 0.7311892494081604, 'support': 1409.0}, 'weighted avg': {'precision': 0.7958916219856498, 'recall': 0.8055358410220014, 'f1-score': 0.7977098840100181, 'support': 1409.0}}
